In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
import random
               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Import the raw Excel flat files

In [ ]:
fy_10 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy10-operating-budget.xlsx")
fy_11 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy11-operating-budget.xlsx")
fy_12 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy12-operating-budget.xlsx")
fy_13 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy13-operating-budget.xlsx")
fy_14 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy14-operating-budget.xlsx")
fy_15 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx")
fy_16 = pd.read_excel("fy16-budget-vs-actuals-revenue-and-expenses.xls")

#Clean up the formating of FY16
fy_16.columns = fy_16.loc[0].values
fy_16 = fy_16.loc[1:]


In [13]:
fy_list[6]['Fiscal Year'] = fy_list[6]['Fiscal Year'].astype(int)
fy_list[6]['Fund Id'] = fy_list[6]['Fund Id'].astype(int)
fy_list[6]['Business Area'] = fy_list[6]['Business Area'].astype(int)
fy_list[6]['Fund Center Id'] = fy_list[6]['Fund Center Id'].astype(int)
fy_list[6]['GL Account'] = fy_list[6]['GL Account'].astype(int)
fy_list[6]['GL Category'] = fy_list[6]['GL Category'].astype(int)
fy_list[6]['Original Budget'] = fy_list[6]['Original Budget'].astype(int)
fy_list[6]['Current Budget'] = fy_list[6]['Current Budget'].astype(float)
fy_list[6]['Actuals'] = fy_list[6]['Actuals'].astype(float)

dtypes_df = pd.DataFrame()

for i, y in enumerate(fy_list):
    dtypes_df[str(i)+"_dtype"] =fy_list[i].dtypes.values
    
for i, y in enumerate(fy_list):
    dtypes_df[str(i)] =fy_list[i].dtypes.index



# Get a list of  common rows across all seven years from 2010-2016

In [14]:
fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]

unique_items_list = []
for index, year in enumerate(fy_list):
    year_items = []
    for group in fy_list[index].groupby(['Fund Id',
           'Business Area', 'Fund Center Id', 'GL Account','GL Description', 'GL Category',
           'GL Category Description',
           'Revenue or Expenditure']):
        year_items.append(group[0])
    if index==0:
        unique_items_list=list(set(year_items))
    else:
        unique_items_list =list(set(year_items) & set(unique_items_list))
        


                                          
unique_items = pd.DataFrame()
unique_items['key']= unique_items_list
unique_items.to_csv('unique_items_16.csv')
len(unique_items)


12342

In [24]:
#

key_df = pd.DataFrame()
key_lists = [[],[],[],[],[],[],[],[]]

for index, row in unique_items.iterrows():
    if len(row['key'].split(","))==8:
        key_lists[0].append(int(row['key'][1:-1].split(',')[0]))   #Fund ID
        key_lists[1].append(int(row['key'][1:-1].split(',')[1]))   #Business Area
        key_lists[2].append(int(row['key'][1:-1].split(',')[2]))   #Fund Center ID
        key_lists[3].append(int(row['key'][1:-1].split(',')[3]))   #GL Account
        key_lists[4].append(row['key'][1:-1].split(',')[4].replace("'","").strip())  #GL Description
        key_lists[5].append(int(row['key'][1:-1].split(',')[5]))                      #GL Category
        key_lists[6].append(row['key'][1:-1].split(',')[6].replace("'","").strip())  #GL Category Description
        key_lists[7].append(row['key'][1:-1].split(',')[7].replace("'","").strip())  #Revenue or Expenditure
    else:
        if len(row['key'].split(","))==9:
            list_ = row['key'].split(",")[:4] + [row['key'].split(",")[4].strip().replace("'","")+"," +" "+ row['key'].split(",")[5].strip().replace("'","")] + row['key'].split(",")[6:]
            list_ = [int(list_[0][1:])]+[int(list_[1])]+[int(list_[2])]+[int(list_[2])]+ [list_[4]] + [int(list_[5])] + [list_[6].replace("'","").strip()] + [list_[7][:-1].replace("'","").strip()]

            for index, item in enumerate(key_lists):
                key_lists[index].append(list_[index])
            

key_df['Fund Id'] = key_lists[0]
key_df['Business Area'] = key_lists[1]
key_df['Fund Center Id'] = key_lists[2]
key_df['GL Account'] = key_lists[3]
key_df['GL Description'] = key_lists[4]
key_df['GL Category'] = key_lists[5]
key_df['GL Category Description'] = key_lists[6]
key_df['Revenue or Expenditure'] = key_lists[7]
                        

In [25]:
for i, y in enumerate(fy_list):
    fy_list[i] = key_df.merge(y, on =['Fund Id',
           'Business Area', 'Fund Center Id', 'GL Account','GL Description', 'GL Category',
           'GL Category Description',
           'Revenue or Expenditure'])

#removed = ['Commitment Set Description', 'Fund Center Name', 'Fund Type Descr', 'Business Area Name', 'Fund Name']
#essentials = fy_16_merged.drop(removed, axis=1).columns

for i, y in enumerate(fy_list):
    print(len(y))

In [29]:
for index, year in enumerate(fy_list):
    fy_list[index]['OA_Difference'] = year['Actuals']-year['Original Budget']
    fy_list[index] = year.loc[year['Revenue or Expenditure']=="Expenditures"]
    fy_list[index].dropna(inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [30]:
for i, y in enumerate(fy_list):
    print(len(y))

11499
11499
11499
11499
11499
11499
11499


In [31]:
for index, year in enumerate(fy_list):
    #print(index)
     #if in 2010 lookback zero years
    if index==0:
        
        fy_list[index]['3Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_Past_Actuals']="999999999999999"
        fy_list[index]['1Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_1Y_Actuals_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['1Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['1Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = "999999999999999"
      
    #if in 2011 lookback one year
    elif index==1:
        fy_list[index]['3Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_Past_Actuals']="999999999999999"
        fy_list[index]['1Y_Past_Actuals']=fy_list[index-1]['Actuals']
        fy_list[index]['2Y_1Y_Actuals_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['1Y_Past_Original_Budget']=fy_list[index-1]['Original Budget']
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['1Y_Past_OA_Difference']=fy_list[index-1]['OA_Difference']
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = "999999999999999"
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = "999999999999999"
    
    #if in 2012 lookback two years
    elif index==2:
        fy_list[index]['3Y_Past_Actuals']="999999999999999"
        fy_list[index]['2Y_Past_Actuals']=fy_list[index-2]['Actuals']
        fy_list[index]['1Y_Past_Actuals']=fy_list[index-1]['Actuals']
        hold_df = fy_list[index][["2Y_Past_Actuals","1Y_Past_Actuals"]]
        fy_list[index]['2Y_1Y_Actuals_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_Original_Budget']="999999999999999"
        fy_list[index]['2Y_Past_Original_Budget']=fy_list[index-2]['Original Budget']
        fy_list[index]['1Y_Past_Original_Budget']=fy_list[index-1]['Original Budget']
        hold_df = fy_list[index][["2Y_Past_Original_Budget","1Y_Past_Original_Budget"]]
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = "999999999999999"
        
        fy_list[index]['3Y_Past_OA_Difference']="999999999999999"
        fy_list[index]['2Y_Past_OA_Difference']=fy_list[index-2]['OA_Difference']
        fy_list[index]['1Y_Past_OA_Difference']=fy_list[index-1]['OA_Difference']
        hold_df = fy_list[index][["2Y_Past_OA_Difference","1Y_Past_OA_Difference"]]
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = "999999999999999"
        
    #otherwise lookback three years
    else:
        print("else, ", index)
        fy_list[index]['3Y_Past_Actuals']=fy_list[index-3]['Actuals']
        fy_list[index]['2Y_Past_Actuals']=fy_list[index-2]['Actuals']
        fy_list[index]['1Y_Past_Actuals']=fy_list[index-1]['Actuals']
        hold_df_2 = fy_list[index][["2Y_Past_Actuals","1Y_Past_Actuals"]]
        hold_df_3 = fy_list[index][["3Y_Past_Actuals","2Y_Past_Actuals","1Y_Past_Actuals"]]
        fy_list[index]['2Y_1Y_Actuals_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Actuals_Mean'] = hold_df_2.mean(axis=1)
        
        fy_list[index]['3Y_Past_Original_Budget']=fy_list[index-3]['Original Budget']
        fy_list[index]['2Y_Past_Original_Budget']=fy_list[index-2]['Original Budget']
        fy_list[index]['1Y_Past_Original_Budget']=fy_list[index-1]['Original Budget']
        hold_df_2 = fy_list[index][["2Y_Past_Original_Budget","1Y_Past_Original_Budget"]]
        hold_df_3 = fy_list[index][["3Y_Past_Original_Budget","2Y_Past_Original_Budget","1Y_Past_Original_Budget"]]
        fy_list[index]['2Y_1Y_Original_Budget_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Original_Budget_Mean'] = hold_df_2.mean(axis=1)
        
        fy_list[index]['3Y_Past_OA_Difference']=fy_list[index-3]['OA_Difference']
        fy_list[index]['2Y_Past_OA_Difference']=fy_list[index-2]['OA_Difference']
        fy_list[index]['1Y_Past_OA_Difference']=fy_list[index-1]['OA_Difference']
        hold_df_2 = fy_list[index][["2Y_Past_OA_Difference","1Y_Past_OA_Difference"]]
        hold_df_3 = fy_list[index][["3Y_Past_OA_Difference","2Y_Past_OA_Difference","1Y_Past_OA_Difference"]]
        fy_list[index]['2Y_1Y_OA_Difference_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_OA_Difference_Mean'] = hold_df_2.mean(axis=1)
        

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

else,  3
else,  4
else,  5
else,  6


In [32]:
houston_data = pd.read_csv("Houston Yearly Data - Sheet1 (1).csv")
houston_data_columns = houston_data.iloc[1,1:].values
houston_data_index = houston_data.iloc[2:,0].values
houston_data  = houston_data.iloc[2:,1:]

houston_data.index= houston_data_index
houston_data.columns = houston_data_columns
houston_data.columns = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019']
houston_data.fillna(0, inplace=True)
houston_data['2019'] = houston_data['2019'].astype(int)

categorical_variables = ['Fund Name', 'Fund Type Descr', 'Business Area Name','Commitment Set Description']



In [33]:
def add_houston_data(year_df, houston_data, houston_data_index):
    for col in houston_data_index:
        year_df[col] = 0
    
    budget_year = year_df.iloc[0]['Fiscal Year']
    previous_year =  year_df.iloc[0]['Fiscal Year'] - 1
    
    year_df['Rockets Playoff Rounds'] = float(houston_data[str(previous_year)]['Rockets Playoff Rounds'].replace(',',''))
    year_df['Rockets Total Attendance'] = float(houston_data[str(previous_year)]['Rockets Total Attendance'].replace(',',''))
    year_df['Texans Average Attendance'] = float(houston_data[str(budget_year)]['Texans Average Attendance'].replace(',',''))
    year_df['Texans Playoff Rounds'] = float(houston_data[str(budget_year)]['Texans Playoff Rounds'].replace(',',''))
    year_df['Astros Yearly Attendance'] = float(houston_data[str(previous_year)]['Astros Yearly Attendance'].replace(',',''))
    year_df['Astros Playoff Rounds'] = float(houston_data[str(previous_year)]['Astros Playoff Rounds'].replace(',',''))

    year_df['Yearly Crime Index'] = float(houston_data[str(previous_year)]['Yearly Crime Index'].replace(',',''))
    year_df['Yearly Avg High Temp'] = float(houston_data[str(previous_year)]['Yearly Avg High Temp'].replace(',',''))
    year_df['Yearly Avg Low Temp'] = float(houston_data[str(previous_year)]['Yearly Avg Low Temp'].replace(',',''))
    year_df['Yearly Rain Fall'] = float(houston_data[str(previous_year)]['Yearly Rain Fall'].replace(',',''))
    
    return year_df



In [46]:
for i, y in enumerate(fy_list):
    print(i, y.shape)

0 (11499, 107)
1 (11499, 107)
2 (11499, 107)
3 (11499, 107)
4 (11499, 107)
5 (11499, 107)
6 (11499, 107)


In [36]:
for i, year in enumerate(fy_list):
    dummies = pd.get_dummies(fy_list[i][categorical_variables]) # One-hot encoding of categorical variables
    year = pd.concat([fy_list[i], dummies], axis=1)
    #year.drop('Fund Center Name', axis=1, inplace=True) #This is repeated information
    fy_list[i] = add_houston_data(year, houston_data, houston_data_index)
    
#shared_columns = set(fy_list[0].columns) & set(fy_list[0].columns)


#drop_columns = list(set(shared_columns).symmetric_difference(set(fy_list[6].columns)))
#fy_list[6].drop(drop_columns, axis=1, inplace=True)
    
    






In [44]:
for i, yi in enumerate(fy_list):
    for j, yi in enumerate(fy_list):
        list_=[x for x in set(fy_list[i].columns) if x not in (set(fy_list[i].columns) & set(fy_list[j].columns))]
        print(i,j,"\n", fy_list[i].shape, fy_list[j].shape, "\n",len(list_), "\n",list_,"\n")
        
        for col in list_:
            fy_list[j][col] = 0
        

0 0 
 (11499, 105) (11499, 105) 
 0 
 [] 

0 1 
 (11499, 105) (11499, 105) 
 0 
 [] 

0 2 
 (11499, 105) (11499, 105) 
 0 
 [] 

0 3 
 (11499, 105) (11499, 105) 
 0 
 [] 

0 4 
 (11499, 105) (11499, 105) 
 0 
 [] 

0 5 
 (11499, 105) (11499, 105) 
 1 
 ['Business Area Name_Houston Police Department'] 

0 6 
 (11499, 105) (11499, 104) 
 3 
 ['Business Area Name_Houston Police Department', 'Business Area Name_Information Technology (IT)', 'Business Area Name_Municipal Court Judicial'] 

1 0 
 (11499, 105) (11499, 105) 
 0 
 [] 

1 1 
 (11499, 105) (11499, 105) 
 0 
 [] 

1 2 
 (11499, 105) (11499, 105) 
 0 
 [] 

1 3 
 (11499, 105) (11499, 105) 
 0 
 [] 

1 4 
 (11499, 105) (11499, 105) 
 0 
 [] 

1 5 
 (11499, 105) (11499, 106) 
 0 
 [] 

1 6 
 (11499, 105) (11499, 107) 
 0 
 [] 

2 0 
 (11499, 105) (11499, 105) 
 0 
 [] 

2 1 
 (11499, 105) (11499, 105) 
 0 
 [] 

2 2 
 (11499, 105) (11499, 105) 
 0 
 [] 

2 3 
 (11499, 105) (11499, 105) 
 0 
 [] 

2 4 
 (11499, 105) (11499, 105) 
 0 


In [45]:
for i, yi in enumerate(fy_list):
    for j, yi in enumerate(fy_list):
        list_=[x for x in set(fy_list[i].columns) if x not in (set(fy_list[i].columns) & set(fy_list[j].columns))]
        print(i,j,"\n", fy_list[i].shape, fy_list[j].shape, "\n",len(list_), "\n",list_,"\n")

0 0 
 (11499, 107) (11499, 107) 
 0 
 [] 

0 1 
 (11499, 107) (11499, 107) 
 0 
 [] 

0 2 
 (11499, 107) (11499, 107) 
 0 
 [] 

0 3 
 (11499, 107) (11499, 107) 
 0 
 [] 

0 4 
 (11499, 107) (11499, 107) 
 0 
 [] 

0 5 
 (11499, 107) (11499, 107) 
 0 
 [] 

0 6 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 0 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 1 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 2 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 3 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 4 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 5 
 (11499, 107) (11499, 107) 
 0 
 [] 

1 6 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 0 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 1 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 2 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 3 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 4 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 5 
 (11499, 107) (11499, 107) 
 0 
 [] 

2 6 
 (11499, 107) (11499, 107) 
 0 
 [] 

3 0 
 (11499, 107) (11499, 107) 
 0 
 [] 

3 1 
 (11499, 107) (11499, 107) 
 0 
 [] 

3 2 
 (1149

In [49]:

#fy_list[6].drop(removed, axis=1, inplace=True)
for index, item in enumerate(fy_list):
    fy_list[index].to_csv('fy_1'+str(index)+'_augmented'+'.csv')
    